In [ ]:
import pandas as pd

file_path = '/content/combined_data.csv'

try:
    questions_df = pd.read_csv(file_path)
    display(questions_df)
except FileNotFoundError:
    print(f"File not found. Please check the path: {file_path}")
except Exception as e:
    print(f"Error reading file: {e}")

,query,original_query,class,most_relevant_segment,respond,function_to_call
0,What are the main topics covered in the NLP cl...,What are the main topics covered in this course?,csci544,Course Description: \nThe course delves into N...,The main topics covered in this course include...,RAG
1,What is the policy on late submissions for ass...,What is the policy on late submissions for ass...,csci544,Late Policy:\nLate Day Policies: Students are ...,Students are allowed a total of 6 late days fo...,RAG
2,What is the required reading material for natu...,"What is the required reading material, and whe...",csci544,Textbooks and materials:\nTextbook1: Jurafsky ...,The required reading materials include three t...,RAG
3,"For the nlp class, how will the course project...","How will the course project be structured, and...",csci544,Course Project: \nStudents will work in teams ...,The course project is a team assignment involv...,RAG
4,How is the final grade distributed across diff...,How is the final grade distributed across diff...,csci544,Grading:\nQuizzes: 10%\nHomework: 20%\nPaper P...,Grades are allocated across several categories...,RAG
...,...,...,...,...,...,...
210,Will csci699 involve any competitions or chall...,Will there be any competitions or challenges?,csci699,Course Description: Competitions and challenge...,Competitions and challenges are occasionally i...,RAG
211,What is expected in assignments for Robotic Pe...,What is expected in assignments?,csci699,"Homework: Assignments: Three assignments, each...","There are three assignments, each worth 15% of...",RAG
212,How is the project graded in csci699?,How is the project graded?,csci699,Grading: Project: 50% of the grade.,The project accounts for 50% of the grade.,RAG
213,What data analysis techniques are used in Robo...,What data analysis techniques are used?,csci699,Course Description: The course focuses on stat...,The course uses state-of-the-art methods for r...,RAG


In [ ]:
!nvidia-smi
!nvcc --version

Sun Nov 24 06:56:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -q einops
!pip install -q FlagEmbedding
!pip install -q peft
!pip install -q faiss-gpu
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.9/347.9 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 100.3 MB/s eta 0:00:00

In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from numpy.linalg import norm
from FlagEmbedding import FlagModel
from FlagEmbedding import FlagReranker
from sentence_transformers import SentenceTransformer

import os
import torch
import faiss
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def load_models(embed_model_name=None, rerank_model_name=None):

    # 如果未提供模型名称，设置默认的嵌入模型和重排序模型名称
    embed_model_name = embed_model_name or 'BAAI/bge-base-en-v1.5'
    rerank_model_name = rerank_model_name or 'BAAI/bge-reranker-v2-m3'

    # 加载嵌入模型
    embed_model = AutoModel.from_pretrained(embed_model_name, trust_remote_code=True, torch_dtype=torch.float32)
    embed_tokenizer = AutoTokenizer.from_pretrained(embed_model_name, trust_remote_code=True)
    embed_model.to(device)
    embed_model.eval()

    # 加载重排序模型
    rerank_model = AutoModelForSequenceClassification.from_pretrained(rerank_model_name, torch_dtype='auto', trust_remote_code=True)
    rerank_tokenizer = AutoTokenizer.from_pretrained(rerank_model_name)
    rerank_model.to(device)
    rerank_model.eval()

    return embed_model, embed_tokenizer, rerank_model, rerank_tokenizer

# 定义可用的嵌入模型字典
available_embed_models = {
    '1': 'BAAI/bge-base-en-v1.5',                      # BGE
    '2': 'jinaai/jina-embeddings-v2-base-en',          # Jina
    '3': 'dunzhang/stella_en_400M_v5',                 # Stella + BGE Reranker
    '4': 'dunzhang/stella_en_400M_v5',                 # Stella + Jina Reranker
    '5': 'sentence-transformers/all-roberta-large-v1'  # Roberta
}

# 定义可用的重排序模型字典
available_rerank_models = {
    '1': 'BAAI/bge-reranker-v2-m3',                             # BGE Reranker
    '2': 'jinaai/jina-reranker-v2-base-multilingual',           # Jina Reranker
    '3': 'BAAI/bge-reranker-v2-m3',                             # Stella + BGE Reranker
    '4': 'jinaai/jina-reranker-v2-base-multilingual',           # Stella + Jina Reranker
    '5': 'BAAI/bge-reranker-v2-m3'                              # Roberta + BGE Reranker
}

print("Please select the embedding model to load:")
print("1: BGE + BGE Reranker")
print("2: Jina + Jina Reranker")
print("3: Stella + BGE Reranker")
print("4: Stella + Jina Reranker")
print("5: Roberta + BGE Reranker")

user_choice = input("Enter a number (1-5): ").strip()

# 获取用户选择的嵌入模型和对应的重排序模型
selected_embed_model = available_embed_models.get(user_choice)
selected_rerank_model = available_rerank_models.get(user_choice)

# 加载选择的模型
embed_model, embed_tokenizer, rerank_model, rerank_tokenizer = load_models(embed_model_name=selected_embed_model, rerank_model_name=selected_rerank_model)

Please select the embedding model to load:
1: BGE + BGE Reranker
2: Jina + Jina Reranker
3: Stella + BGE Reranker
4: Stella + Jina Reranker
5: Roberta + BGE Reranker
Enter a number (1-5): 1


In [ ]:
courses_numbers = ['544', '566', '585', '596', '599', '626', '677', '699']
course_names = ['Applied Natural Language Processing', 'Deep Learning and its Applications', 'Database Systems', 'Scientific Computing and Visualization',
         'Distributed Systems', 'Text as Data', 'Advanced Computer Vision', 'Robotic Perception']

name_to_num = dict(zip(course_names, courses_numbers))

In [ ]:
# 对文本进行编码的函数
def encode(texts):
    inputs = embed_tokenizer(texts, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        embeddings = embed_model(**inputs).last_hidden_state.mean(dim=1)
        # 转换为float32
    return embeddings.to(torch.float32)

# 读取txt文件并按"\n\n"切割 -> 自行修改！！！
def read_and_split_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    segments = content.split('\n\n##')

    return segments


def compute_similarity(query, segments):
    query_embedding = encode([query])[0].cpu()
    segment_embeddings = encode(segments).cpu()

    query_embedding = query_embedding / norm(query_embedding)
    segment_embeddings = segment_embeddings / norm(segment_embeddings, axis=1, keepdims=True)

    # 计算余弦相似度
    similarities = query_embedding @ segment_embeddings.T

    # 将分数、query和文档分段组合并排序
    results = sorted(zip(similarities.tolist(), segments), key=lambda x: x[0], reverse=True)

    return results

In [ ]:
def generate_embeddings(texts):
    # 使用 embed_tokenizer 和 embed_model 生成嵌入（适用于 BGE、Stella 等模型）
    inputs = embed_tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        embeddings = embed_model(**inputs).last_hidden_state.mean(dim=1).cpu().numpy().astype('float32')
    return embeddings

def build_vector_database(course_names, user_choice):
    print("Starting to build vector database...")

    if user_choice == '2':  # 如果选择的是 Jina 模型
        print("Using Jina model to encode course names...")
        course_embeddings = embed_model.encode(course_names)  # 直接使用 embed_model.encode 方法生成嵌入
    else:  # 如果是 BGE、Stella 等其他模型
        print("Using generate_embeddings to encode course names...")
        course_embeddings = generate_embeddings(course_names)  # 调用 generate_embeddings 生成嵌入

    # 转为 float32 并进行 L2 归一化
    np_course_embeddings = np.array(course_embeddings).astype('float32')
    faiss.normalize_L2(np_course_embeddings)


    # 创建 FAISS 索引，选择内积搜索方式 (dot-product search)
    index_innerproduct = faiss.IndexFlatIP(np_course_embeddings.shape[1])
    print("FAISS index created on CPU.")

    # 将已归一化的嵌入添加到 CPU 索引中
    #index_innerproduct.add(np_course_embeddings)
    #print("Embeddings added to CPU index.")

    #return index_innerproduct  # 返回 CPU 上的索引##

    ## 将索引从 CPU 移动到 GPU
    res = faiss.StandardGpuResources()  # 创建 GPU 资源
    gpu_index = faiss.index_cpu_to_gpu(res, 0, index_innerproduct)  # 将索引移动到 GPU

    # 将已归一化的嵌入添加到 GPU 索引中
    gpu_index.add(np_course_embeddings)
    print("Embeddings added to GPU index.")

    return gpu_index  # 返回 GPU 上的索引

# 搜索最相似的课程
def search(query, database, embed_dim, topk=1):
    # 判断使用哪种方法生成查询嵌入
    if user_choice == '2':  # Jina 模型
        query_embed = embed_model.encode([query])[0]
    else:  # 其他模型，如 BGE、Stella 等
        query_embed = generate_embeddings([query])[0]

    query_embed = np.array(query_embed).astype('float32')

    _, idx = database.search(query_embed.reshape((1, embed_dim)), topk)  # (768,) -> (1, 768)
    idx = idx.reshape(-1)

    ret = [course_names[i] for i in idx]
    return ret


database = build_vector_database(course_names, user_choice)

Starting to build vector database...
Using generate_embeddings to encode course names...
FAISS index created on CPU.
Embeddings added to GPU index.


In [ ]:
def find_course(query, database):
    for num in courses_numbers:
        if f"csci{num}" in query.lower():
            return f"csci{num}"

    for num in courses_numbers:
        if num in query:
            return f"csci{num}"
    for name, course_num in name_to_num.items():
        if name.lower() in query.lower():
            return f"csci{course_num}"

    result = search(query, database, embed_dim=database.d, topk=1)[0]
    if result in courses_numbers:
        return f"csci{result}"
    else:
        return f"csci{name_to_num.get(result, 'Course not found')}"

In [ ]:
import csv
from collections import Counter

# 从 CSV 文件中读取查询并存储
def load_queries_from_csv(file_path):
    queries = []
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            if len(row) < 6:
                print(f"Skipping incomplete row: {row}")
                continue

            if any(cell.strip().lower() in ["n/a", "nah"] for cell in row):
                #print(f"Skipping invalid question row: {row}")
                continue

            # 提取有效的列数据
            query = row[0].strip()
            original_query = row[1].strip()
            course_class = row[2].strip()
            most_relevant_segment = row[3].strip()
            respond = row[4].strip()
            function_to_call = row[5].strip()
            queries.append((query, original_query, course_class, most_relevant_segment, respond, function_to_call))

    return queries

def extract_first_line(segment):
    # 检查段落是否包含 `##`，并提取 `##` 后面的第一行内容
    lines = segment.strip().split('\n')
    if lines[0].startswith("##"):
        return lines[1].strip() if len(lines) > 1 else ""  # 提取 `##` 后的第一行内容
    else:
        return lines[0].strip()  # 没有 `##`，返回整个段落的第一行


# 执行检索并获取 top-k 结果
def RAG(queries, segments, k=3):
    retrieved_segments = []

    for query_tuple in queries:
        query = query_tuple[1] # 使用original_query作为检索项
        course_class = query_tuple[2]
        most_relevant_segment = query_tuple[3].split(':')[0]  # 提取冒号前的内容
        all_similarity = compute_similarity(query, segments)

        # 按相似度筛选段落
        results = [segment for score, segment in all_similarity if score > 0.6]

        # 如果有足够相关的段落，将其重新排序并获取前 k 个
        if results:
            sentence_pairs = [[query, segment] for segment in results]

            final_results = []
            for pair in sentence_pairs:
                inputs = rerank_tokenizer(pair[0], pair[1], return_tensors='pt', truncation=True, max_length=1024).to(device)
                with torch.no_grad():
                    outputs = rerank_model(**inputs)
                    score = outputs.logits.cpu().float().numpy().flatten()[0]

                    final_results.append((score, pair[1]))

            # 排序并选择 top k
            final_results.sort(key=lambda x: x[0], reverse=True)
            top_k_results = [extract_first_line(pair[1]) for pair in final_results[:k]]
        else:
            top_k_results = ['!!!Failed to find any segment!!!']

        # 将前 k 个结果存储到 retrieved_segments 列表中
        retrieved_segments.append(top_k_results)

    return retrieved_segments

In [ ]:
# 加载课程段落和查询
course_ranges = {
    'csci544': '/content/CSCI544.txt',
    'csci566': '/content/CSCI566.txt',
    'csci585': '/content/CSCI585.txt',
    'csci596': '/content/CSCI596.txt',
    'csci599': '/content/CSCI599.txt',
    'csci626': '/content/CSCI626.txt',
    'csci677': '/content/CSCI677.txt',
    'csci699': '/content/CSCI699.txt'
}

# 确保从 CSV 加载的 queries 列表不为空
queries = load_queries_from_csv('/content/combined_data.csv')
if not queries:
    print("No valid queries found in CSV.")
else:
    print(f"Loaded {len(queries)} valid queries.")

# 执行检索并获取 top-k 段落
all_courses_queries = []
all_retrieved_segments = []
k = 3

for course, file_path in course_ranges.items():
    print(f"Processing course {course}...")
    segments = read_and_split_file(file_path)

    # get all queries for this course
    course_queries = [query for query in queries if query[2] == course]
    print(f"Number of queries for {course}: {len(course_queries)}")

    # 对每一门课执行 RAG
    retrieved_segments = RAG(course_queries, segments, k)
    all_courses_queries.extend(course_queries)
    all_retrieved_segments.extend(retrieved_segments)

Loaded 215 valid queries.
Processing course csci544...
Number of queries for csci544: 29
Processing course csci566...
Number of queries for csci566: 28
Processing course csci585...
Number of queries for csci585: 27
Processing course csci596...
Number of queries for csci596: 27
Processing course csci599...
Number of queries for csci599: 28
Processing course csci626...
Number of queries for csci626: 27
Processing course csci677...
Number of queries for csci677: 27
Processing course csci699...
Number of queries for csci699: 22


In [ ]:
def evaluate_rag(all_courses_queries, all_retrieved_segments, k=3):
    matched = 0
    for idx, courses_query in enumerate(all_courses_queries):
        query = courses_query[1]
        course_class = courses_query[2]
        most_relevant_segment = courses_query[3].split(':')[0]
        top_k_results = all_retrieved_segments[idx][:k]

        # 检查 Expected Segment 是否包含在 Top 3 Segments 中
        if any(most_relevant_segment in res for res in top_k_results):
            matched += 1

        # 打印课程名称和实际查询内容和前 k 个结果
        #print(f"Course Class: {course_class}")
        #print(f"Query {idx + 1} ({query}): Top {k} Segments: {top_k_results}")
        #print(f"Expected Segment from CSV: {most_relevant_segment}")
        #print()

    accuracy = matched / len(all_courses_queries)
    #print(f"Expected Segment included in Top {k} Segments: {accuracy:.2%}")

    return accuracy

# Analyze and print the results
top_3_acc = evaluate_rag(all_courses_queries, all_retrieved_segments, k=3)
print(f"Top-3 Accuracy: {top_3_acc:.2%}")

top_1_acc = evaluate_rag(all_courses_queries, all_retrieved_segments, k=1)
print(f"Top-1 Accuracy: {top_1_acc:.2%}")


Top-3 Accuracy: 88.84%
Top-1 Accuracy: 80.93%
